### 1. Load data

In [12]:
import os
import pandas as pd
import numpy as np
from data_prep import store_categorical_metadata

In [2]:
#load orignal data
# file_path = os.path.join(os.curdir,'data','Kopie van BastianLewisData.xlsx')
# df = pd.read_excel(file_path,sheet_name='Result 1')

In [15]:
#load latest data
data_set_name = "StrevonExport"
file_path = os.path.join(os.curdir,'data','StrevonExport.xlsx')
df = pd.read_excel(file_path,sheet_name='StrevonExport')

In [16]:
#Drop irrelevant variables
var_file_path = os.path.join(os.curdir,'data','Kopie van BastianLewisData.xlsx')
df_vars = pd.read_excel(var_file_path,sheet_name='VariableSelection')
var_drop = list(df_vars['Name'][df_vars['Status'] == 'Drop'])

df.drop(labels = var_drop,  axis= 1, inplace = True)

### 2. Select relevant candidates/rows

In [17]:
# alleen 'prioriteit' = 1 houden & geen afwijsBasisGegevens (isnull) 
print('unique prioriteit values:',  df['prioriteit'].unique())
print('unique afwijsBasisGegevens values:',  df['afwijsBasisGegevens'].unique())

df = df[df['prioriteit'] == 1]
df = df[pd.isnull(df['afwijsBasisGegevens'])]
df.drop(labels = ['prioriteit','afwijsBasisGegevens'],  axis= 1, inplace = True)
print(f"number of candidates left: {len(df)}")
print(df['starter'].value_counts())

unique prioriteit values: [nan  3.  1.  2.  4.]
unique afwijsBasisGegevens values: [nan 'postal_code' 'drivers_license' 'age' 'dutch_language_skill']
number of candidates left: 4088
0    3946
1     142
Name: starter, dtype: int64


In [18]:
df_backup = df.copy()

### 3. clean dataset

In [19]:
from data_prep import run_data_prep
data_dir = "data"

In [20]:
df = run_data_prep(df, data_dir)

### 4. Clean categorical variables

In [21]:
# Cleaning values with low frequency <10
def clean_categorical_variable(df: pd.DataFrame, column_name: str):
    print(f"working on {column_name}")
    print(f"{len(df[df[column_name].isna()])} missings")
    print("recategorizing all values that appear less then 10 times to 'other'")

    value_counts = df[column_name].value_counts()
    values_to_replace = value_counts[value_counts < 10].index
    values_to_keep = value_counts[value_counts >= 10].index
    df[column_name] = df[column_name].apply(lambda x: 'other' if x in values_to_replace else x)
    print(f"replaced the following values with other: {values_to_replace}")
    print(f"Keeping the following values: {values_to_keep}")
    print("_________________________")
    return df


def prep_dataset_for_modelling(df, data_set_name):
    # Identify columns with object data type (categorical variables)
    categorical_columns = df.select_dtypes(include=['object']).columns
    for cat_col in categorical_columns:
        df = clean_categorical_variable(df, cat_col)

    store_categorical_metadata(df, f"data/{data_set_name}_categorical_meta_data.json")
    # Identify non-object type columns
    non_categorical_columns = df.select_dtypes(exclude=['object']).columns

    # Dummy code the categorical columns
    df_dummies = pd.get_dummies(df[categorical_columns])

    # Select non-categorical columns
    df_non_categorical = df[non_categorical_columns]

    # Concatenate the dummy coded variables with the non-categorical columns
    df_final = pd.concat([df_non_categorical, df_dummies], axis=1)
    return df_final

In [22]:
df_final = prep_dataset_for_modelling(df, data_set_name)

working on recruiter
555 missings
recategorizing all values that appear less then 10 times to 'other'
replaced the following values with other: Index(['Jurrian Sturm', 'Stavros Tatanidis', 'Wim van Ginkel',
       'Gwyneth van Diermen', 'Fadoua Beghiyl', 'Gideon Den Admirant'],
      dtype='object')
Keeping the following values: Index(['Job van den Berg', 'Jermaine Symons', 'Lennart Huibers',
       'Isabelle Walschots', 'Richard Felix Omondi', 'Wim Van Ginkel',
       'Job Bruinsma'],
      dtype='object')
_________________________
working on utm_source
172 missings
recategorizing all values that appear less then 10 times to 'other'
replaced the following values with other: Index(['8kvl4ikw535', 'l', 'ecosia', 'fb-ig-web', 'duckduckgo', 'linktr',
       'com', 'googleadservices', 'nl', 'youtube', 'm',
       'nationale vacature gids'],
      dtype='object')
Keeping the following values: Index(['google', 'strevon', 'fb-ig', 'facebook', 'indeed', 'undefined',
       'nationaleberoepengi

In [24]:
df_final.to_csv(os.path.join(data_dir,f"{data_set_name}_final.csv"))

### cleaning and merging utm_source and source

In [ ]:
column = "utm_source"
plot_categorical_distribution(df, column)

In [ ]:
column = "source"
plot_categorical_distribution(df, column)

In [ ]:
def clean_and_combine_source_columns(df):
    df['source'] = df['source'].str.lower()
    df['source'] = df['source'].replace("nationale beroepengids", "nationaleberoepengids")
    df['source'] = df['source'].replace("werkenbijstrevon", "strevon")
    df['source'] = df['source'].replace("onbekend", "undefined")
    df['source'] = df['source'].fillna(df['utm_source'])
    clean_categorical_variable(df, 'source')

    df['utm_source'] = df['utm_source'].str.lower()
    df["utm_source"].replace('werkenbijstrevon', 'strevon',inplace=True) #Mag dit is dit hetzelfde?
    df['source'] = df['source'].replace("direct","strevon")
    df['utm_source'] = df['utm_source'].replace("direct","strevon")
    df['utm_source'] = df['utm_source'].fillna(df['source'])
    clean_categorical_variable(df, 'utm_source')

    column_a = 'utm_source'
    column_b = 'source'
    x = "undefined"
    df[column_a] = df.apply(lambda row: row[column_b] if row[column_a] == x else row[column_a], axis=1)

    column_a = 'source'
    column_b = 'utm_source'
    x = "undefined"
    df[column_a] = df.apply(lambda row: row[column_b] if row[column_a] == x else row[column_a], axis=1)

    column_a = 'source'
    column_b = 'utm_source'
    x = "facebook"
    df[column_a] = df.apply(lambda row: row[column_b] if row[column_a] == x else row[column_a], axis=1)

    ## Drop source and keep utm_source
    df.drop(['source'], axis=1, inplace=True)
    return df

df = clean_and_combine_source_columns(df)

In [ ]:
column = "utm_source"
plot_categorical_distribution(df, column)

### Visualizing, cleaning and merging utm_medium and medium

In [ ]:
print("BEFORE")
column = "utm_medium"
plot_categorical_distribution(df, column)

column = "medium"
plot_categorical_distribution(df, column)

def merging_two_columns(df, column_a, column_b):
    # Visualizeing, Cleaning and merging utm_medium and medium
    df[column_b] = df[column_b].str.lower()
    df[column_b] = df[column_b].replace("onbekend", "undefined")

    df[column_a] = df[column_a].str.lower()
    df[column_a] = df[column_a].replace("onbekend","undefined")
    df[column_a] = df[column_a].fillna(df[column_b])

    x = "undefined"
    df[column_a] = df.apply(lambda row: row[column_b] if row[column_a] == x else row[column_a],
                            axis=1)

    df.drop(column_b, axis=1, inplace = True)
    return df

df = merging_two_columns(df, "utm_medium", "medium")
print("AFTER")
column = "utm_medium"
plot_categorical_distribution(df, column)

### Visualize, Clean and Merge utm_campaign and campagneNaam

In [ ]:
column = "utm_campaign"
plot_categorical_distribution(df, column)

In [ ]:
column = "campagneNaam"
plot_categorical_distribution(df, column)

In [ ]:
def clean_utm_campaign(df):
    # cleaning utm_campaign
    column_name = 'utm_campaign'
    df[column_name].replace('installatiemonteur_amstedram', 'installatiemonteur_amsterdam', inplace=True) #Mag dit is dit hetzelfde?
    df[column_name].replace('installatiemonteur_amsterdan', 'installatiemonteur_amsterdam', inplace=True) #Mag dit is dit hetzelfde?
    
    installatiemonteur_values = [val for val in df['utm_campaign'].unique() if "installatiemonteur-" in str(val)]
    installatiemonteur_values.remove('installatiemonteur-exp')
    for val in installatiemonteur_values:
        df[column_name].replace(val, 'installatiemonteur', inplace=True)

    elektromonteur_values = [val for val in df['utm_campaign'].unique() if "elektromonteur-" in str(val)]
    elektromonteur_values += ["elektromonteur50215728-102-1"]
    for val in elektromonteur_values:
        df[column_name].replace(val, 'elektromonteur', inplace=True)

    installatietechniek_values = [val for val in df['utm_campaign'].unique() if "installatietechniek-" in str(val).lower()]
    for val in installatietechniek_values:
        df[column_name].replace(val, 'installatietechniek', inplace=True)

    elektrotechniek_values = [val for val in df['utm_campaign'].unique() if "elektrotechniek-" in str(val).lower()]
    for val in elektrotechniek_values:
        df[column_name].replace(val, 'elektrotechniek', inplace=True)
    return df

In [ ]:
df = clean_utm_campaign(df)

In [ ]:
column = "utm_campaign"
plot_categorical_distribution(df, column)

In [ ]:
def split_campaign_locations(df):
    #create location of utm_campaign variable
    df["utm_campaign_location"] = df["utm_campaign"].apply(lambda x: str(x).split("_")[-1] if len(str(x).split("_")) > 0 else None)
    locations=['amsterdam','utrecht','dordrecht','leiden','gouda','denhaag','utrecht','rotterdam']
    df["utm_campaign_location"] = df["utm_campaign_location"].apply(lambda x: x if x in locations else None)
    #remove locations from utm_campaigns
    df["utm_campaign_no_loc"] = df["utm_campaign"].apply(lambda x: "_".join(str(x).split("_")[:-1]) if str(x).split("_")[-1] in locations else x)
    return df

In [ ]:
df = split_campaign_locations(df)

In [ ]:
df = clean_categorical_variable(df, "utm_campaign_no_loc")

In [ ]:
column = "utm_campaign_location"
plot_categorical_distribution(df, column)

In [ ]:
df['utm_campaign'].value_counts()

In [ ]:
column = "utm_campaign_no_loc"
plot_categorical_distribution(df, column)

In [ ]:
column = "utm_campaign"
plot_categorical_distribution(df, column)

In [ ]:
# CampagneNaam is same as utm_campaign so drop
df.drop(['campagneNaam'], axis=1, inplace=True)

## Visualize, clean and merge utm_adgroup and adgroup

In [ ]:
column = "utm_adgroup"
plot_categorical_distribution(df, column)

In [ ]:
column = "adgroup"
plot_categorical_distribution(df, column)

In [ ]:
def split_adgroup_locations(df):
    #create location of utm_adgroup variable
    column_name = "utm_adgroup"
    df["utm_adgroup_location"] = df[column_name].apply(lambda x: str(x).split("_")[-1] if len(str(x).split("_")) > 0 else None)
    locations=['amsterdam','utrecht','dordrecht','leiden','gouda','denhaag','utrecht','rotterdam']
    df["utm_adgroup_location"] = df["utm_adgroup_location"].apply(lambda x: x if x in locations else None)
    #remove locations from utm_campaigns
    df[column_name+"_no_loc"] = df[column_name].apply(lambda x: "_".join(str(x).split("_")[:-1]) if str(x).split("_")[-1] in locations else x)
    df = clean_categorical_variable(df, column_name+"_no_loc")
    column_name = "utm_adgroup"
    df = clean_categorical_variable(df, column_name)
    return df

In [ ]:
df = split_adgroup_locations(df)

In [ ]:
column = "utm_adgroup"
plot_categorical_distribution(df, column)

In [ ]:
df['utm_adgroup'].value_counts()

### cleaning referrer

In [ ]:
def clean_referrer_column(df):    
    values = ['strevon.nl', 'www.google.com', 'direct', 'm.facebook.com', np.nan,
        'www.google.nl', 'www.nationaleberoepengids.nl',
        'www.adsensecustomsearchads.com', 'nl.indeed.com',
        'www.ecosia.org', 'duckduckgo.com', 'www.bing.com', 'linktr.ee',
        'www.google.de', 'www.google.com.tr', 'www.google.ae',
        'www.google.it', '8kvl4ikw535.typeform.com', 'www.google.co.jp',
        'l.facebook.com', 'undefined', 'www.youtube.com', 'l.wl.co',
        'com.google.android.googlequicksearchbox',
        'www.googleadservices.com', 'www.google.co.za', 'typeform.com',
        'www.google.fr', 'werkenbijstrevon.nl', 'l.instagram.com',
        'nl.search.yahoo.com', 'lm.facebook.com', 'www.tiktok.com',
        'instagram.com', 'android-app:', 'www.google.ch',
        'strevon-nl.webpkgcache.com', 'web.telegram.org',
        'syndicatedsearch.goog']

    google_values = [val for val in values if 'google' in str(val)]
    facebook_values = [val for val in values if 'facebook' in str(val)]
    strevon_values = [val for val in values if 'strevon' in str(val)] + ["direct"]

    df['referrer'] =  df['referrer'].apply(lambda x: 'google' if str(x) in google_values else x)
    df['referrer'] =  df['referrer'].apply(lambda x: 'facebook' if str(x) in facebook_values else x)
    df['referrer'] =  df['referrer'].apply(lambda x: 'strevon' if str(x) in strevon_values else x)
    clean_categorical_variable(df, 'referrer')


    df['referrer'] = df['referrer'].str.replace(".com",'')
    df['referrer'] =df['referrer'].str.replace("www.",'')
    df['referrer'] =df['referrer'].str.replace("nl.",'')
    df['referrer'] =df['referrer'].str.replace(".nl",'')
    return df

In [ ]:
df = clean_referrer_column(df)

In [ ]:
column = "referrer"
plot_categorical_distribution(df, column)

## Conversiepunt Visualize if needed clean

In [ ]:
column = 'conversiepunt'
df[column] = df[column].str.lower()
plot_categorical_distribution(df, column)

## Pagina visualize and clean if needed 

In [ ]:
column = 'pagina'
plot_categorical_distribution(df, column)

In [ ]:
def convert_pagina_to_parent_page(df):
    column = 'pagina'
    df[column] = df[column].apply(lambda x: x.strip("/") if type(x)==str else x)
    df['parent_page'] = df[column].apply(lambda x: x.split("/")[0] if type(x)==str else x)
    return df

In [ ]:
df = convert_pagina_to_parent_page(df)

In [ ]:
len(df['parent_page'].unique())

In [ ]:
column = 'parent_page'
plot_categorical_distribution(df,column)

## UitkomstTelefonischContact en UitkomstTelefonischDeal Visualize and Clean

In [ ]:
def merge_UitkomstTelefonisch(df):
    df['uitkomstTelefonischDeal'] = df['uitkomstTelefonischDeal'].fillna(df["uitkomstTelefonischContact"])
    df.drop(column, axis=1, inplace=True)
    return df

In [ ]:

column = "uitkomstTelefonischDeal"

In [ ]:

plot_categorical_distribution(df,column)


## Visualizing and Cleaning leeftijd

In [ ]:
df['leeftijd'].unique()

In [ ]:
import random

def categorize(x):
    if (x>0) & (x<31):
        return "18-30 jaar"
    elif (x>30) & (x<50):
        return "31-49 jaar"
    elif x > 49:
        return "50 jaar of ouder"
    else:
        return np.nan

def cleaning_leeftijd(df):
    df['leeftijd'] = df['leeftijd'].apply(lambda x: str(random.choice(range(18,30))) if x=="18-30 jaar" else x)
    df['leeftijd'] = df['leeftijd'].apply(lambda x: str(random.choice(range(31,49))) if x=="31-49 jaar" else x)
    df['leeftijd'] = df['leeftijd'].apply(lambda x: str(random.choice(range(31,45))) if x=="31-45 jaar" else x)
    df['leeftijd'] = df['leeftijd'].apply(lambda x: str(random.choice(range(50,65))) if x=="50 jaar of ouder" else x)
    df['leeftijd'] = df['leeftijd'].apply(float)
    df['leeftijd'] = df['leeftijd'].replace(0,np.nan)
    df['leeftijd'] = df['leeftijd'].replace(1,np.nan)

    df['leeftijd_cat'] = df['leeftijd'].apply(lambda x: categorize(x))
    return df


In [ ]:
df = cleaning_leeftijd(df)

In [ ]:
column = 'leeftijd_cat'
plot_categorical_distribution(df,column)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

def plot_histogram(df, column, bins=30):
    plt.figure(figsize=(10, 6))
    sns.histplot(df[column], bins=bins, kde=False, color='blue')
    plt.title(f'Histogram of {column}')
    plt.xlabel(column)
    plt.ylabel('Frequency')
    plt.show()

def plot_density(df, column):
    plt.figure(figsize=(10, 6))
    sns.kdeplot(df[column], shade=True, color='purple')
    plt.title(f'Density Plot of {column}')
    plt.xlabel(column)
    plt.ylabel('Density')
    plt.show()

def plot_boxplot(df, column):
    plt.figure(figsize=(10, 6))
    sns.boxplot(y=df[column], color='green')
    plt.title(f'Box Plot of {column}')
    plt.xlabel('Category')
    plt.ylabel(column)
    plt.show()

def plot_violinplot(df, column):
    plt.figure(figsize=(10, 6))
    sns.violinplot(y=df[column], color='orange')
    plt.title(f'Violin Plot of {column}')
    plt.xlabel('Category')
    plt.ylabel(column)
    plt.show()

plot_histogram(df, 'leeftijd')
plot_density(df, 'leeftijd')
plot_boxplot(df, 'leeftijd')
plot_violinplot(df, 'leeftijd')


## Ben je in het bezit van een rijbewijs

In [ ]:
def clean_rijbewijs(df):
    column ="Ben je in het bezit van rijbewijs?"
    df[column] = df[column].str.lower()
    df[column] = df[column].str.strip(",")
    return df
df = clean_rijbewijs(df)
df[column].value_counts()

## beschikking tot eigen vervoer?

In [ ]:
def clean_eigen_vervoer(df):
    column = "beschikking tot eigen vervoer?"
    df[column] = df[column].str.lower()
    df[column] = df[column].apply(lambda x: "nee dit heb ik niet" if str(x) in ["geen auto","geen vervoer"] else x)
    df[column] = df[column].apply(lambda x: "ja een eigen auto of motor" if "motor" in str(x) else x)
    df[column] = df[column].apply(lambda x: "ja een eigen auto of motor" if "auto" in str(x) else x)
    df[column] = df[column].apply(lambda x: "ja een eigen auto of motor" if str(x)=="ja" else x)
    df[column] = df[column].apply(lambda x: "nee dit heb ik niet" if str(x)=="nee" else x)
    return df
df = clean_eigen_vervoer(df)
df["beschikking tot eigen vervoer?"].value_counts()

## Score 1

In [ ]:
def clean_score_1(df):
    column = 'score 1'
    answer_options = ["Laminaat leggen, lampen aansluiten op elektra, sleutelen aan scooter/auto",
                    "Gordijnen ophangen, batterij vervangen rookmelder, lampje verwisselen",
                    "Meer dan een half jaar ervaring in een technische functie (bijvoorbeeld monteur)"]
    df[column] = df[column].apply(lambda x: x if x in answer_options else np.nan)
    return df

df = clean_score_1(df)

In [ ]:
plot_categorical_distribution(df, column)

In [ ]:
column = 'score 2'
df[column].value_counts()
#plot_categorical_distribution(df, column)

In [ ]:
plot_categorical_distribution(df, column)

In [ ]:
column = 'score 3'
df[column].value_counts()
#plot_categorical_distribution(df, column)

In [ ]:
plot_categorical_distribution(df, column)

## Jaar ervaring

In [ ]:
from datetime import datetime
import re 

def replace_fun(str_val):
     replace_dict = {
         ',':'.',
         ' ':'',
         'half':'0.5',
         'een': '1',
         'twee': '2',
         'drie': '3',
         'vier': '4',
         'vijf': '5',
         'zes': '6',
         'zeven': '7',
         'acht': '8',
         'negen': '9',
         'months':'maanden',
         'jaren':'jaar'
     }
     for orig, new in replace_dict.items():
         str_val = str_val.replace(orig, new)
         
     return str_val 

def extract_vals(str_val):
    try:
        #re.match(r'^-?\d+(?:\.\d+)$', str_val) is None: # geen float
        val = float(str_val)
        return val
    except:
        if str_val in ['niet','geen']:
            val = 0
            return val
        elif 'sinds' in str_val or 'vanaf' in str_val:
            matches = re.findall('(\d{4})',str_val)
            if len(matches) == 0:
                val = np.nan
            else:
                val = float(matches[0])
            return val
        if 'jaar' in str_val:
            pattern = r'(\d+\.\d+|\d+)\s*jaar'
        elif 'maand' in str_val:
            pattern = r'(\d+\.\d+|\d+)\s*maand'
        else:
            val = np.nan
            return val
        matches = re.findall(pattern, str_val)
        if len(matches) == 0:
            val = np.nan
        else:
            numbers = [float(match) for match in matches]
            val = numbers[0]
            if 'maand' in str_val:
                val = val/12
        
        return val

# Calculate the number of years worked
def calculate_years_worked(row):
    start_year = row['jaar_ervaring']
    end_date = row['cdate']
    
    if isinstance(end_date, datetime):
        piet= 0
    else:
        end_date = datetime(2024,1,1)
    # If start_year is a float, separate the year and the fraction
    year = int(start_year)
    fraction = start_year - year
    
    # Calculate the number of full years
    years_worked = end_date.year - year
    
    # Adjust for the fraction of the year
    if fraction > 0:
        # Calculate the exact date by adding the fraction as months
        start_date = datetime(year, 1, 1) + pd.DateOffset(months=int(fraction * 12))
        if end_date < start_date:
            years_worked -= 1
        elif end_date > start_date:
            extra_months = (end_date - start_date).days / 365.25
            years_worked += extra_months
    
    return years_worked


def bereken_jaar_ervaring(df):
    df.loc[df['Hoe lang in dienst/werkloos'] == 'Vijf een half jaar', 'Hoe lang in dienst/werkloos'] = '5.5jaar'

    df['jaar_ervaring'] = df['Hoe lang in dienst/werkloos'].apply(lambda x: extract_vals(replace_fun(x.lower())) if isinstance(x, str) else x)
    df.loc[df['jaar_ervaring'] > 1950, 'jaar_ervaring'] = df[df['jaar_ervaring'] > 1950].apply(calculate_years_worked,axis=1)
    return df

df = bereken_jaar_ervaring(df)

In [ ]:
plot_density(df, 'jaar_ervaring')

In [ ]:
df['jaar_ervaring'].sort_values()

## Convert postcode

In [ ]:
def convert_postcode(df):
    # postcode > stad, provincie, randstad ja/nee
    df['postcode_getal'] = df['postcode'].str[:4]
    postal_code_data = pd.read_excel('data/postcodesNL.xlsx',converters={'Postcode':str})
    df = df.merge(postal_code_data, left_on='postcode_getal', right_on='Postcode', how='left')

    randstad_list = ['Amsterdam','Rotterdam','Den Haag','Utrecht','Almere','Haarlem','Amersfoort','Zaanstad','Haarlemmermeer','Zoetermeer','Leiden','Dordrecht','Alphen aan den Rijn','Westland','Alkmaar','Delft']
    df['randstad'] = df['Gemeente'].isin(randstad_list)
    df.loc[df['Gemeente'].isna(), 'randstad'] = np.nan
    df.loc[df['postcode'] == 'Overig', 'randstad']=False
    df.drop(['postcode'], axis=1, inplace=True)
    return df

In [ ]:
df = convert_postcode(df)

In [ ]:
plot_categorical_distribution(df,'Gemeente')

In [ ]:
plot_categorical_distribution(df,'Provincie')

In [ ]:
plot_categorical_distribution(df,'randstad')

## Werksituatie

In [ ]:
def clean_werksituatie(df):
       df['Werksituatie'] = df['Werksituatie'].str.lower()
       df['Werksituatie'].replace("werkloos","ik ben werkloos",inplace=True)
       klus_answers = [ 'niks: 0 klussen',
              'weinig: 3 tot 4 klussen', 'regelmatig: 5 tot 8 klussen',
              'bij uitzondering: 1 tot 2 klussen', 'veel: meer dan 8 klussen']
       df['Werksituatie'] = df['Werksituatie'].apply(lambda x: "ik ben zzp'er" if x in klus_answers else x)
       answers = ['ik heb een tijdelijk contract (bepaalde tijd)', 'ik ben werkloos',
              'ik heb een vast contract', "ik ben zzp'er"]
       df['Werksituatie'] = df['Werksituatie'].apply(lambda x: np.nan if x not in answers else x)
       return df

In [ ]:
df = clean_werksituatie(df)

In [ ]:
column = "Werksituatie"
plot_categorical_distribution(df, column)

In [ ]:
df['Voorkeursbranche'] = df['Voorkeursbranche'].str.lower()

In [ ]:
column = "Voorkeursbranche"
plot_categorical_distribution(df, column)

In [ ]:
column = "Strevon startsalaris"
plot_categorical_distribution(df, column)

In [ ]:
def clean_strevon_startsalaris(df):
    answers = ["Dit is haalbaar", "Dit is een uitdaging", "Dit is niet haalbaar"]
    df[column] = df[column].apply(lambda x: np.nan if x not in answers else x)
    return df

In [ ]:
column = "Strevon werktijden"
plot_categorical_distribution(df, column)

In [ ]:
answers = ["Dit is haalbaar", "Dit is een uitdaging", "Dit is niet haalbaar"]
df[column] = df[column].apply(lambda x: np.nan if x not in answers else x)

In [ ]:
column = "groupid"
plot_categorical_distribution(df, column)

In [ ]:
column = "stage"
plot_categorical_distribution(df, column)

In [ ]:
column = "status"
plot_categorical_distribution(df, column)

In [ ]:
df['diff_days'] = (df['belafspraak'] - df['cdate']).dt.days
df[['cdate', 'belafspraak','diff_days']]

In [ ]:
def drop_columns(df):
    dropcols = ["Motivatie", "Huidig/Laatste functie", 'Hoe lang in dienst/werkloos',
                'Meest trotste project', "uitkomstTelefonischContact", "Overname bedrag (W&S)",
                "Verloning marge stopper of W&S overname", "berekening", 'postcode_getal', 'Postcode']
    dropcols = [col for col in dropcols if col in df.columns.to_list()]
    
    for col in dropcols:
        df.drop(col ,axis=1,inplace=True)
    return df


In [ ]:
df = drop_columns(df)

In [ ]:
date_cols = ['cdate','geboortedatum','belafspraak']
for col in date_cols:
    df[col] = pd.to_numeric(df[col])
    #df[col] = df[col].apply(lambda x: float(pd.Timestamp(x)) )

In [ ]:
cols = df.columns.to_list()
cols.remove('starter')

In [ ]:
def prep_dataset_for_modelling(df):
    # Identify columns with object data type (categorical variables)
    categorical_columns = df.select_dtypes(include=['object']).columns
    for cat_col in categorical_columns:
        df = clean_categorical_variable(df, cat_col)

    store_categorical_metadata(df, "data/categorical_meta_data.json")
    # Identify non-object type columns
    non_categorical_columns = df.select_dtypes(exclude=['object']).columns

    # Dummy code the categorical columns
    df_dummies = pd.get_dummies(df[categorical_columns])

    # Select non-categorical columns
    df_non_categorical = df[non_categorical_columns]

    # Concatenate the dummy coded variables with the non-categorical columns
    df_final = pd.concat([df_non_categorical, df_dummies], axis=1)
    return df_final

In [ ]:
from data_prep import store_categorical_metadata


In [ ]:
# Identify columns with object data type (categorical variables)
categorical_columns = df.select_dtypes(include=['object']).columns

# Identify non-object type columns
non_categorical_columns = df.select_dtypes(exclude=['object']).columns

# Dummy code the categorical columns
df_dummies = pd.get_dummies(df[categorical_columns])

# Select non-categorical columns
df_non_categorical = df[non_categorical_columns]

# Concatenate the dummy coded variables with the non-categorical columns
df_final = pd.concat([df_non_categorical, df_dummies], axis=1)

In [ ]:
# # Calculate the sum of each column
# column_sums = df_final.sum()

# # Identify columns where the sum is greater than 10
# columns_to_keep = column_sums[column_sums > 10].index

# # Drop columns that don't sum to more than 10
# df_final_filtered = df_final[columns_to_keep]

In [ ]:
df_final.to_csv("data\df_final_filtered2.csv",index=False)

In [ ]:
len(df_final.columns)

In [ ]:
# Calculate the correlation of each column with the target column 'starter'
correlations = df_final.corrwith(df_final['starter'])

# Create a list of correlations
correlation_list = correlations.drop('starter').tolist()  # Drop the target column itself from the list

# Create a list of column names
column_names = correlations.drop('starter').index.tolist()  # Drop the target column itself from the list

# Combine column names and correlations into a list of tuples
correlation_tuples = list(zip(column_names, correlation_list))
# Sort the list of tuples by the absolute value of the correlation coefficients in descending order
sorted_correlations = sorted(correlation_tuples, key=lambda x: abs(x[1]), reverse=True)

# Display the sorted list of correlations
print("\nSorted list of strongest correlations with 'starter':")
for column, corr in sorted_correlations:
    print(f"{column}: {corr}")

In [ ]:
df_corr = pd.DataFrame(sorted_correlations,columns=['variable','correlation'])
df_corr

In [ ]:
df_corr.iloc[50:100].head(50)

In [ ]:
pd.DataFrame(df.describe(datetime_is_numeric=True))

In [ ]:
df.info()

In [ ]:
pd.DataFrame(df[['cdate','geboortedatum','belafspraak']].describe(datetime_is_numeric=True))

In [ ]:
df['geboortedatum'].iloc[0]

In [ ]:
date_cols = ['cdate','geboortedatum','belafspraak']
for col in date_cols:
    df[col] = pd.to_numeric(df[col])
    #df[col] = df[col].apply(lambda x: float(pd.Timestamp(x)) )

In [ ]:
for col in df.columns.to_list():
    #print(f"{col}") 
    print(f"{len(df[df[col].isna()])}") 